In [4]:
import subprocess
import re
import os

def get_video_dimensions(file_path):
    try:
        # Expand user path
        file_path = os.path.expanduser(file_path)
        
        # Run ffmpeg to get video info
        result = subprocess.run(
            ['ffmpeg', '-i', file_path],
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            text=True
        )
        
        # Extract dimensions from the output
        output = result.stderr
        # print(output)  # Print the output for debugging
        
        # Updated regex pattern to match different formats
        match = re.search(r'Stream.*Video.*? (\d{2,5})x(\d{2,5})', output)
        
        if match:
            width, height = match.groups()
            return int(width), int(height)
        else:
            raise ValueError("Could not find video dimensions")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

file_path = "~/Desktop/fin.mov"
width, height = get_video_dimensions(file_path)
if width and height:
    print(f"The dimensions of the video are {width}x{height} pixels.")
else:
    print("Failed to get the video dimensions.")

The dimensions of the video are 1460x1686 pixels.


In [5]:
import sys
import os 
sys.path.append('/Library/Application Support/Blackmagic Design/DaVinci Resolve/Developer/Scripting/Modules')
import DaVinciResolveScript as dvr_script

resolve = dvr_script.scriptapp("Resolve")

projectManager = resolve.GetProjectManager()
currentProject = projectManager.GetCurrentProject()
media_pool = currentProject.GetMediaPool()

currentTimeline = currentProject.GetCurrentTimeline()
if currentTimeline is None:
    currentTimeline = media_pool.CreateEmptyTimeline("MyTimeLine")

print(f"Current project: {currentProject.GetName()}")

# Insert a simple title into the timeline
simple_title = currentTimeline.InsertTitleIntoTimeline('Text')

# Path to the latest .mov file
latest_mov_file = file_path
latest_mov_file = os.path.expanduser(latest_mov_file)

# Check if the file exists
if os.path.exists(latest_mov_file):
    # Import the media file into the project
    mediaPool = currentProject.GetMediaPool()
    mediaPool.ImportMedia([latest_mov_file])
    
    # Get the imported clip
    mediaItems = mediaPool.GetRootFolder().GetClipList()
    importedClip = None
    for clip in mediaItems:
        if clip.GetClipProperty("File Path") == latest_mov_file:
            importedClip = clip
            break

    if importedClip:
        # Append the imported clip to the timeline
        mediaPool.AppendToTimeline([importedClip])
        print("The imported clip has been added to the timeline.")
    else:
        print("The imported clip was not found in the media pool.")
else:
    print("The specified .mov file does not exist.")

Current project: hi2
The imported clip has been added to the timeline.


In [6]:
# Define the new resolution
sourceWidth = width
sourceHeight = height

# Update project settings
currentProject.SetSetting('timelineResolutionWidth', str(sourceWidth))
currentProject.SetSetting('timelineResolutionHeight', str(sourceHeight))

# Create a new timeline if necessary
if currentTimeline is None:
    currentTimeline = media_pool.CreateEmptyTimeline("MyTimeLine")

# Update timeline settings
currentTimeline.SetSetting('timelineOutputResolutionWidth', str(sourceWidth))
currentTimeline.SetSetting('timelineOutputResolutionHeight', str(sourceHeight))
currentTimeline.SetSetting('timelineResolutionWidth', str(sourceWidth))
currentTimeline.SetSetting('timelineResolutionHeight', str(sourceHeight))
print("hi")

hi
